In [ ]:
#  UserWarningを省略したい場合はこのコードを実行すること
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
%%time
import pandas as pd
import geopandas as gpd
import os
import gc
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
from plotnine import *
plt.rcParams['font.family'] = "Noto Sans CJK JP"

ziko_list = [file for file in os.listdir("/workspace/accident") if file.endswith(".csv")]
bind_data = pd.DataFrame()
for file_name in ziko_list:
    p = os.path.join("/workspace/accident/", file_name)
    csv_data = pd.read_csv(p, encoding='Shift-JIS')
    bind_data = pd.concat([bind_data, csv_data])

bind_data = bind_data[bind_data['地点　経度（東経）'].apply(lambda x: str(x).strip().replace(' ', '').isdigit())]
bind_data = bind_data[bind_data['地点　緯度（北緯）'].apply(lambda x: str(x).strip().replace(' ', '').isdigit())]

lon_list = []
lat_list = []

for _, row in bind_data.iterrows():
    dms = str(row['地点　経度（東経）']).strip().zfill(10)
    degrees = int(dms[:3])
    minutes = int(dms[3:5])
    seconds = int(dms[5:].split('.')[0])
    lon = degrees + (minutes / 60.0) + (seconds /1000/ 3600.0)
    lon_list.append(lon)
    
    dms = str(row['地点　緯度（北緯）']).strip().zfill(9)
    degrees = int(dms[:2])
    minutes = int(dms[2:4])
    seconds = int(dms[4:].split('.')[0])
    lat = degrees + (minutes / 60.0) + (seconds /1000/ 3600.0)
    lat_list.append(lat)

bind_data['lon'] = lon_list
bind_data['lat'] = lat_list

bind_data = bind_data.dropna(subset=['lon', 'lat'])

bind_data = bind_data[(bind_data['lon'] > 120) & (bind_data['lon'] < 155)]
bind_data = bind_data[(bind_data['lat'] > 20) & (bind_data['lat'] < 46)]

bind_data['geometry'] = [Point(xy) for xy in zip(bind_data['lon'], bind_data['lat'])]
ziko = gpd.GeoDataFrame(bind_data, geometry='geometry', crs=4326)

ziko['ID'] = range(1, len(ziko.index) + 1)

re = {10:'北海道',
      11:'北海道',
      12:'北海道',
      13:'北海道',
      14:'北海道',
      20:'青森県',
      21:'岩手県',
      22:'宮城県',
      23:'秋田県',
      24:'山形県',
      25:'福島県',
      30:'東京都',
      40:'茨城県',
      41:'栃木県',
      42:'群馬県',
      43:'埼玉県',
      44:'千葉県',
      45:'神奈川県',
      46:'新潟県',
      47:'山梨県',
      48:'長野県',
      49:'静岡県',
      50:'富山県',
      51:'石川県',
      52:'福井県',
      53:'岐阜県',
      54:'愛知県',
      55:'三重県',
      60:'滋賀県',
      61:'京都府',
      62:'大阪府',
      63:'兵庫県',
      64:'奈良県',
      65:'和歌山県',
      70:'鳥取県',
      71:'島根県',
      72:'岡山県',
      73:'広島県',
      74:'山口県',
      80:'徳島県',
      81:'香川県',
      82:'愛媛県',
      83:'高知県',
      90:'福岡県',
      91:'佐賀県',
      92:'長崎県',
      93:'熊本県',
      94:'大分県',
      95:'宮崎県',
      96:'鹿児島県',
      97:'沖縄県'
     }
ziko['都道府県コード'] = ziko['都道府県コード'].replace(re)
ziko = ziko.to_crs(epsg = 3857)

kihon_list = [file for file in os.listdir("/workspace/polyon/") if file.endswith(".shp")]
for i in kihon_list:
    d = "/workspace/polygon/" + i
    polygon = gpd.read_file(d)
    polygon = polygon.to_crs(epsg=3857)
    t =  polygon['PREF'].iloc[1] + polygon['CITY'].iloc[1]+polygon['PREF_NAME'].iloc[1] + polygon['CITY_NAME'].iloc[1]
    title = polygon['PREF_NAME'].iloc[1] + polygon['CITY_NAME'].iloc[1]
    point = ziko[(ziko['都道府県コード'] == str(polygon['PREF_NAME'].iloc[2])) & (ziko['市区町村コード'] == int(polygon['CITY'].iloc[2]))]
    if len(point) == 0:
        error_path = "/workspace/result/dot/" + t + ".txt"
        with open(error_path, 'w') as f:
            f.write('creating prop is failed:\n')
            f.write('The Number of Point:0') 
    else:
        dot = (ggplot() +
                theme(text=element_text(family='Noto Sans CJK JP'),
                      panel_background=element_rect(fill='white'),  # パネルの背景色を白に
                      plot_background=element_rect(fill='white'),
                      panel_grid_major=element_blank(),  # グリッド線を非表示
                      panel_grid_minor=element_blank(),
                      axis_title=element_blank(),        # 軸タイトルを非表示
                      axis_text=element_blank(),         # 軸目盛りテキストを非表示
                      axis_ticks=element_blank()) +
                geom_map(data = polygon, fill = None, color = 'black') + 
                geom_map(data = point, fill = 'red', color = None, size = 0.5) +
                labs(title=title) +
                coord_equal())
        file_path = "/workspace/result/dot/" + t + ".png"
        dot.save(file_path)
        del dot
        gc.collect()